In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
#from keras.datasets import mnist
import plotly.express as px
from IPython.display import display

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz") #mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [3]:
# Get all the values whos true classification is less than 5
y_train_i = np.nonzero(y_train < 5)[0]
#print(y_train_i)
# Find those grids in x_train
#display(x_train.shape)
x_train = x_train[y_train_i]
#x_train

In [4]:
display(px.imshow(x_train[1]))
px.imshow(x_train[2])


def image_compression(mat):
    n = mat.shape[0] # 28
    new = np.zeros(shape = (int(n/2), int(n/2)))
    for i in range(0, n, 2):
        val = np.mean(mat[np.ix_([i, i+1], [i, i+1])])
        new[i//2, i//2] = val
        #for j in range(i, i+):

    # flatten the matrix by stacking columns 
    new = new.flatten(order = "F")
    return new

example = image_compression(x_train[0])
#print(example)
#print(x_train[0])
#px.imshow(image_compression(x_train[0]))
#x_train[1]

x_train_comp = np.zeros((x_train.shape[0], 196))

#x_train_comp.shape[0]
for i in range(x_train_comp.shape[0]):
    x_train_comp[i,:] = image_compression(x_train[i,:])

In [5]:
x_train_comp[1,:]

array([  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  , 179.5 ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
      

# Problem 3

In [36]:
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.datasets import fetch_openml

#mnist = fetch_openml('mnist_784')
#data = mnist.data

def expectation_maximization(data, num_clusters, max_iterations=100, tolerance=0.01, num_initializations=3):
    best_likelihood = -np.inf
    best_means = None
    best_covariances = None

    for i in range(num_initializations):
        # Randomly initialize the means and covariances
        means = np.random.rand(num_clusters, data.shape[1])
        covariances = np.ones((num_clusters, data.shape[1]))  # assuming spherical Gaussians
        for _ in range(max_iterations):
            # E-step: Compute the responsibilities
            responsibilities = np.zeros((len(data), num_clusters))
            for cluster in range(num_clusters):
                responsibilities[:, cluster] = multivariate_normal.pdf(data, means[cluster], np.diag(covariances[cluster]))
            total_responsibility = np.sum(responsibilities, axis=1)
            responsibilities /= total_responsibility[:, np.newaxis]

            # M-step: Update the means and covariances
            for cluster in range(num_clusters):
                means[cluster] = np.sum(responsibilities[:, cluster][:, np.newaxis] * data, axis=0) / np.sum(responsibilities[:, cluster])
                covariances[cluster] = np.sum(responsibilities[:, cluster][:, np.newaxis] * (data - means[cluster])**2, axis=0) / np.sum(responsibilities[:, cluster])

            # Compute the log-likelihood
            log_likelihood = np.sum(np.log(np.sum(responsibilities, axis=1))) / len(data)

            # Check for convergence
            if log_likelihood - best_likelihood < tolerance:
                break

            # Update the best parameters if a better likelihood is found
            if log_likelihood > best_likelihood:
                best_likelihood = log_likelihood
                best_means = means
                best_covariances = covariances

    return best_means, best_covariances


num_clusters = 5
best_means, best_covariances = expectation_maximization(x_train_comp, num_clusters)

C:\Users\Gianni\AppData\Local\Temp\ipykernel_9728\2964432665.py:23: RuntimeWarning:

invalid value encountered in divide



ValueError: array must not contain infs or NaNs

In [70]:
from scipy import stats

def expectation_maximization(data, num_clusters=5, max_iterations=100, tolerance = 0.0001, num_initializations = 1, seed = 123):
    #np.random.state(seed)

    # Create parameters...randomly
    pi = np.random.uniform(0,1)
    mu0, mu1 = np.random.normal(0,1, size = (data.shape[1],)), np.random.normal(0,1, size = (data.shape[1],))
    mu = np.random.normal(0, 1, size = (num_clusters, data.shape[1]))
    # Spherical Gaussian means we are making a matrix where diagonal is all the same value
    #Sigma = np.eye(N = data.shape[1])
    # Create positive semi-definite matrices
    #Sigma0 = np.random.rand(data.shape[1], data.shape[1])
    #Sigma0 = Sigma0 @ Sigma0.T
    Sigma0 = np.eye(data.shape[1])
    #Sigma1 = np.random.rand(data.shape[1], data.shape[1])
    #Sigma1 = Sigma1 @ Sigma1.T 
    Sigma1 = np.eye(data.shape[1])

    e = e_step(data, pi, mu0, mu1, Sigma0, Sigma1)
    #for i in range(5):
        # E step
        #respon = e_step(data, 5, mu, Sigma0)
        #z0 = np.log(stats.multivariate_normal(mu0, Sigma0).pdf(data))
        #z1 = np.log(stats.multivariate_normal(mu1, Sigma1).pdf(data))

    return e


#def e_step(data, num_clusters, means, covariances):
#    responsibilities = np.zeros(data.shape)
    #for cluster in range(num_clusters):
    #    responsibilities[:, cluster] = multivariate_normal.pdf(data, means[cluster], covariances[cluster])
    #total_responsibility = np.sum(responsibilities, axis=1)
    #responsibilities /= total_responsibility[:, np.newaxis]
    #return responsibilities

def e_step(data, pi, mu0, mu1, Sigma0, Sigma1):
        log_p_y_x = np.log([1-pi, pi])[np.newaxis, ...] + \
                np.log([stats.multivariate_normal(mu0, Sigma0).pdf(data),
            stats.multivariate_normal(mu1, Sigma1).pdf(data)]).T

expectation_maximization(x_train_comp)

C:\Users\Gianni\AppData\Local\Temp\ipykernel_9728\3375629857.py:40: RuntimeWarning:

divide by zero encountered in log



In [73]:
# Define the parameters of the GMM
n_components = 3  # Number of components in the mixture
means = np.array([[0, 0], [4, 4], [7, 7]])  # Means of each component
covariances = np.array([[[1, 0], [0, 1]], [[0.5, 0], [0, 0.5]], [[0.2, 0], [0, 0.2]]])  # Covariances of each component
weights = np.array([0.4, 0.3, 0.3])  # Weights of each component

# Set the random seed for reproducibility
np.random.seed(0)

# Generate samples from the GMM
n_samples = 1000  # Number of samples to generate
component_indices = np.random.choice(n_components, size=n_samples, p=weights)
samples = np.zeros((n_samples, 2))

for i, idx in enumerate(component_indices):
    samples[i] = np.random.multivariate_normal(means[idx], covariances[idx])

X_samples=samples
X_samples.size

2000

In [58]:
#import silence_tensorflow.auto
import tensorflow as tf 
import tensorflow_probability as tfp 

from tqdm import tqdm # progress meter

def em(data, n_classes, n_iterations, random_seed = 1001):
    #Set seed for reproducible results
    np.random.seed(random_seed)

    n_samples = data.shape[0]
    
    # We estabilish our parameters with random generation, n_classes = clusters 
    mu = np.random.rand(n_classes)
    Sigma = np.random.rand(n_classes)
    pi = np.random.dirichlet(np.ones(n_classes))

    
    for em_iter in range(n_iterations):
        # E-Step
        rsblty = np.zeros((n_samples, n_classes))

        #for row in range(n_samples):
        #   for col in range(n_classes):
        #        rsblty[row, col] = pi[row] * tfp.distributions.Normal(loc=mu[col], scale = Sigma[col]).prob(data[row])

        rsblty = tfp.distributions.Normal(loc=mu, scale = Sigma).prob(data.reshape(-1, 1)).numpy()

        #Normalize responsibilities
        rsblty = rsblty / np.linalg.norm(rsblty, axis = 1, ord =1, keepdims=True) 

        # Use nan sums to avoid any nan's (treat them like 0)
        class_rsblty = np.nansum(rsblty, axis = 0)

        # M steps
        for col in range(n_classes):
            # Updating rules
            pi[col] = class_rsblty[col] / n_samples
            mu[col] = np.sum(rsblty[:, col] * data) / class_rsblty        
            Sigma[col] = np.sqrt(np.sum(rsblty[:, col] * (data - mu[col])**2) / class_rsblty)
        # M-Step
    return class_rsblty
    
em(x_train_comp[0:5], 5, 1)

C:\Users\Gianni\AppData\Local\Temp/ipykernel_48820/1979457293.py:30: RuntimeWarning:

invalid value encountered in true_divide



ValueError: operands could not be broadcast together with shapes (980,) (5,196) 

$ \sum_{i=1}^{n}\log \left[\sum_{j=1}^{k} p_{t}(x_i, Z_i=j)\right] $